# Simple Trading Rule

In [ ]:
import os
os.chdir('c:\\Users\\Danie\\OneDrive\\Documents\\Coding\\pysystemtrade\\pysystemtrade')
os.getcwd()

In [10]:
from sysdata.sim.csv_futures_sim_data import csvFuturesSimData
import matplotlib.pyplot as plt
%matplotlib inline

Work up a minimum example of a trend following system

In [3]:
import sys
import os

# Add the parent directory of pysystemtrade to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

In [4]:
from sysdata.sim.csv_futures_sim_data import csvFuturesSimData
import matplotlib.pyplot as plt
%matplotlib inline

Let's get some data

We can get data from various places; however for now we're going to use
prepackaged 'legacy' data stored in csv files

In [ ]:
data = csvFuturesSimData()
data

In [ ]:

# First check available instruments
print(data.get_instrument_list())

# Then use an available instrument, for example SP500
instrument_code = 'SP500'  # Changed from 'EDOLLAR'
price = data.daily_prices(instrument_code)
ewmac = calc_ewmac_forecast(price, 32, 128)
ewmac.columns = ['forecast']
ewmac.tail(5)

// ... existing code ...

We get stuff out of data with methods

In [ ]:
# First check available instruments
print(data.get_instrument_list())

# Then use SP500 which we know is available
instrument_code = 'SP500'
price = data.daily_prices(instrument_code)
print(price.tail(5))

In [ ]:
print(data.get_instrument_list())
print(data.get_raw_price("EDOLLAR").tail(5))

data can also behave in a dict like manner (though it's not a dict)

In [ ]:
data['SP500']

In [ ]:
data.keys()

... however this will only access prices
(note these prices have already been backadjusted for rolls)

We have extra futures data here

In [ ]:
data.get_instrument_raw_carry_data("JPY").tail(6)

Technical note: csvFuturesSimData inherits from FuturesData which itself inherits from simData
The chain is 'data specific' <- 'asset class specific' <- 'generic'

Let's create a simple trading rule

No capping or scaling

In [13]:
import pandas as pd
from sysquant.estimators.vol import robust_vol_calc


def calc_ewmac_forecast(price, Lfast, Lslow=None):
    """
    Calculate the ewmac trading rule forecast, given a price and EWMA speeds
    Lfast, Lslow and vol_lookback

    """
    # price: This is the stitched price series
    # We can't use the price of the contract we're trading, or the volatility
    # will be jumpy
    # And we'll miss out on the rolldown. See
    # https://qoppac.blogspot.com/2015/05/systems-building-futures-rolling.html

    price = price.resample("1B").last()

    if Lslow is None:
        Lslow = 4 * Lfast

    # We don't need to calculate the decay parameter, just use the span
    # directly
    fast_ewma = price.ewm(span=Lfast).mean()
    slow_ewma = price.ewm(span=Lslow).mean()
    raw_ewmac = fast_ewma - slow_ewma
    vol = robust_vol_calc(price.diff())
    return raw_ewmac / vol


Try it out

(this isn't properly scaled at this stage of course)


In [ ]:
# Test the EWMAC strategy
instrument_code = 'JPY'  # Changed from EDOLLAR
price = data.daily_prices(instrument_code)
ewmac = calc_ewmac_forecast(price, 32, 128)
ewmac.columns = ['forecast']
ewmac.tail(5)

In [ ]:
ewmac.plot();
plt.title('Forecast')
plt.ylabel('Position')
plt.xlabel('Time')


Did we make money?


In [ ]:
from systems.accounts.account_forecast import pandl_for_instrument_forecast
account = pandl_for_instrument_forecast(forecast=ewmac, price = price)
account.curve().plot();
plt.title('Profit and Loss')
plt.ylabel('PnL')
plt.xlabel('Time');

In [ ]:
account.percent.stats()

In [1]:
import os 
# Now run the prebaked systems example
os.chdir('c:\\Users\\Danie\\OneDrive\\Documents\\Coding\\pysystemtrade\\pysystemtrade\\examples\\introduction')

In [17]:
import sys
import os

# Add the parent directory of pysystemtrade to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

In [ ]:
# Now try the prebaked system
from systems.provided.example.simplesystem import simplesystem

my_system = simplesystem()
print(my_system)

In [ ]:
# Let's look at positions for SOFR
print(my_system.portfolio.get_notional_position("SOFR").tail(5))


In [ ]:
# Create the data object first
from sysdata.sim.csv_futures_sim_data import csvFuturesSimData

# Initialize the data
data = csvFuturesSimData()

# Now let's try with config and data
from sysdata.config.configdata import Config

my_config = Config("systems.provided.example.simplesystemconfig.yaml")
my_system = simplesystem(config=my_config, data=data)  # Using the data we already loaded
print(my_system.portfolio.get_notional_position("SOFR").tail(5))

In [ ]:
# Let's try the more sophisticated chapter 15 system
from systems.provided.futures_chapter15.basesystem import futures_system

system = futures_system()
print("Portfolio Sharpe Ratio:", system.accounts.portfolio().sharpe())

# Plot the equity curve
system.accounts.portfolio().curve().plot()
plt.title('Chapter 15 System Performance')
plt.ylabel('Portfolio Value')
plt.xlabel('Time')
plt.show()


In [ ]:
# Get account statistics for the chapter 15 system
system.accounts.portfolio().percent.stats()

In [ ]:
# For a specific instrument (e.g., "SOFR")
system.accounts.pandl_for_instrument("SOFR").percent.stats()

In [ ]:
# Let's cache the system for faster future use
system.cache.pickle("private.this_system_name.pck")

# Delete and reload to demonstrate caching
del system
system = futures_system()
system.cache.unpickle("private.this_system_name.pck")

# This will run much faster now
print("Portfolio Sharpe Ratio (from cache):", system.accounts.portfolio().sharpe())